In [59]:
import numpy as np
import pandas as pd
from datetime import datetime

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers

from xgboost import XGBRegressor, XGBRFRegressor

In [60]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [61]:
%load_ext tensorboard
train_data = pd.read_csv("train.csv",index_col = 'id')
test_data = pd.read_csv("test.csv",index_col = 'id')

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [62]:
train_data = train_data.fillna(train_data.mean())
test_data = test_data.fillna(train_data.mean())

In [63]:
Y_train = train_data['loss'].copy()
X_train = train_data.copy().drop('loss', axis = 1)

In [64]:
X_train.describe()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99
count,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,...,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000
mean,0.511213,51.378476,0.107155,0.050010,3595.133426,8.205953,164.508753,0.375533,16.669745,1.190382,...,0.581002,4856.812768,22.579100,2.030554,0.079692,1.555097,2.417556,0.537484,1.576900,8.048805
std,0.307884,42.396636,1.322200,0.792368,6072.401061,5.475723,183.335563,0.813597,99.758709,0.099700,...,0.445022,8501.609009,14.849390,0.900211,0.587780,9.253785,0.892563,0.226589,0.646306,5.647368
min,-0.069273,-17.000000,-7.895580,-1.475560,-7589.280000,-3.291050,-40.967200,-4.143080,-502.813000,0.934037,...,-0.379189,-12695.700000,-4.059170,0.057800,-1.998800,-24.686300,-1.131980,0.005249,-0.646967,-0.842397
25%,0.251287,18.000000,-0.611172,-0.719418,163.864750,4.110127,27.894900,-0.026245,-17.392025,1.132640,...,0.280511,73.203100,11.525450,1.471650,-0.408975,-4.004925,1.906718,0.359646,1.215810,3.732800
50%,0.514962,41.000000,0.253815,0.004099,943.000500,7.472445,91.005250,0.619862,8.714945,1.170370,...,0.480568,1060.025000,19.993200,1.660830,0.215710,0.759942,2.340430,0.531347,1.451285,7.182205
75%,0.777323,75.000000,0.759249,0.765456,4115.355000,11.030950,240.843750,0.933855,55.407625,1.218880,...,0.766532,5572.982500,32.271625,2.320085,0.503134,6.202502,2.910020,0.709807,1.901633,10.998550
max,1.072070,273.000000,9.768590,1.680190,37847.500000,35.078000,947.143000,4.010380,465.956000,1.712450,...,3.422660,54334.600000,79.912400,5.403020,1.944190,42.890400,5.576040,1.105400,4.492620,34.019200


In [65]:
X_train[X_train.columns] = X_train[X_train.columns].apply(lambda x: ((x - x.min()) / (x.max() - x.min())))
test_data[test_data.columns] = test_data[test_data.columns].apply(lambda x: ((x - x.min()) / (x.max() - x.min())))

In [66]:
model = keras.Sequential([
        layers.Input(shape = (100,)),
        #layers.Embedding(400, 8, embeddings_regularizer='l2', input_length=75),
        #layers.Conv1D(32, 3, activation='relu', padding='same', input_shape=(100,1)),
        #layers.Flatten(),
        #layers.Dropout(0.2),
        #layers.LayerNormalization(),
        layers.Dense(units = 64, activation ='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(1, activation = 'relu'),
    ])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 64)                6464      
_________________________________________________________________
batch_normalization_4 (Batch (None, 64)                256       
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 65        
Total params: 6,785
Trainable params: 6,657
Non-trainable params: 128
_________________________________________________________________


In [67]:
X_train_split, X_val_split, Y_train_split, Y_val_split = train_test_split(X_train, Y_train, test_size = 0.2, random_state = 100, stratify = Y_train)

In [68]:
early_stopping = callbacks.EarlyStopping(
    monitor="root_mean_squared_error",
    patience=20,
    min_delta=0.0001,
    restore_best_weights=True,
)

In [69]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir="logs/" + datetime.now().strftime("%Y%m%d-%H%M%S"))

#model.compile(loss='mse', optimizer = keras.optimizers.Adam(learning_rate=0.0005),  metrics=[tf.keras.metrics.RootMeanSquaredError()])
#history = model.fit(X_train_split, Y_train_split,
#          batch_size = 256, epochs = 100,
#          validation_data=(X_val_split, Y_val_split),
#          callbacks=[early_stopping,tensorboard_callback],
#           )
#score = model.evaluate(X_val_split, Y_val_split, verbose = 0)
#print('Test loss: {}'.format(score[0]))

kfold = KFold(n_splits=5, shuffle=True)

for train_index, test_index in kfold.split(X_train, Y_train):
    model.compile(loss='mse', optimizer = keras.optimizers.Adam(learning_rate=0.05), metrics=[tf.keras.metrics.RootMeanSquaredError()])
    history = model.fit(X_train.iloc[train_index], Y_train.iloc[train_index],
          batch_size = 256, epochs = 100,
          validation_data=(X_train.iloc[test_index], Y_train.iloc[test_index]),
          callbacks=[early_stopping,tensorboard_callback],
           )
    score = model.evaluate(X_train.iloc[test_index], Y_train.iloc[test_index], verbose = 0)
    print('Test loss: {}'.format(score))

In [70]:
kfold = KFold(n_splits=5, shuffle=False)

for train_index, test_index in kfold.split(X_train, Y_train):
    model = XGBRegressor(n_estimators=1500, learning_rate=0.01, n_jobs=8, verbosity=1)
    model.fit(X_train.iloc[train_index], Y_train.iloc[train_index],
                 early_stopping_rounds=5,
                 eval_set=[(X_train.iloc[test_index], Y_train.iloc[test_index])],
                 verbose=True)

[0]	validation_0-rmse:10.08995
[1]	validation_0-rmse:10.05091
[2]	validation_0-rmse:10.01256
[3]	validation_0-rmse:9.97478
[4]	validation_0-rmse:9.93756
[5]	validation_0-rmse:9.90100
[6]	validation_0-rmse:9.86505
[7]	validation_0-rmse:9.82974
[8]	validation_0-rmse:9.79498
[9]	validation_0-rmse:9.76071
[10]	validation_0-rmse:9.72713
[11]	validation_0-rmse:9.69403
[12]	validation_0-rmse:9.66147
[13]	validation_0-rmse:9.62947
[14]	validation_0-rmse:9.59800
[15]	validation_0-rmse:9.56704
[16]	validation_0-rmse:9.53660
[17]	validation_0-rmse:9.50669
[18]	validation_0-rmse:9.47728
[19]	validation_0-rmse:9.44840
[20]	validation_0-rmse:9.41995
[21]	validation_0-rmse:9.39189
[22]	validation_0-rmse:9.36442
[23]	validation_0-rmse:9.33742
[24]	validation_0-rmse:9.31092
[25]	validation_0-rmse:9.28477
[26]	validation_0-rmse:9.25915
[27]	validation_0-rmse:9.23389
[28]	validation_0-rmse:9.20916
[29]	validation_0-rmse:9.18482
[30]	validation_0-rmse:9.16083
[31]	validation_0-rmse:9.13733
[32]	validation

In [71]:
test_data[test_data.columns] = test_data[test_data.columns].apply(lambda x: ((x - x.min()) / (x.max() - x.min())))
predicts = model.predict(test_data)

In [72]:
output = pd.DataFrame(predicts, columns = ['loss'])
output['id'] = test_data.index
output.to_csv('submission.csv', index=False)



